In [36]:
import pandas as pd
import numpy as np
from tqdm import tqdm

terminology definition

we say drug era B is a **subsequent drug era** for drug era A if:
1. drug era A and drug era B are with the same person, and
2. drug_era_end_date of B > drug_era_start_date of A (i.e. B starts after A ends)

we say drug era B is the **closest subsequent drug era** for drug era A if:
- after A ends, B starts first among all the subsequent drug eras \
(theoretically, for one drug era, there could be multiple closest subsequent drug eras)

In [37]:
df = pd.read_csv("../../dataset/dataset.tsv", sep="\t")
# df.head()
# display(df.loc[:, df.columns != "eid"].head())

In [41]:
def get_drug_switches_all(df: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Find the closest subsequent drug era B for each drug era A.
    Output both valid and invalid drug eras.
    """
    # Ensure datetime format
    if df["drug_era_start_date"].dtype != "datetime64[ns]":
        df["drug_era_start_date"] = pd.to_datetime(
            df["drug_era_start_date"], format="%d/%m/%Y"
        )
    if df["drug_era_end_date"].dtype != "datetime64[ns]":
        df["drug_era_end_date"] = pd.to_datetime(
            df["drug_era_end_date"], format="%d/%m/%Y"
        )

    # Sort all data by eid and start date
    df_sorted = df.sort_values(["eid", "drug_era_start_date"]).reset_index(drop=True)

    # Identify single drug era patients
    eid_counts = df_sorted.groupby("eid").size()
    single_era_eids = eid_counts[eid_counts == 1].index
    single_era_df = df_sorted[df_sorted["eid"].isin(single_era_eids)].copy()
    single_era_df["reason"] = "single_drug_era"

    # Process multi-drug era patients
    multi_era_df = df_sorted[~df_sorted["eid"].isin(single_era_eids)]

    # Create cross join within eid groups
    pairs = pd.merge(
        multi_era_df.assign(key=1),
        multi_era_df.assign(key=1),
        on=["eid", "key"],
        suffixes=("_A", "_B"),
    ).drop("key", axis=1)

    # Calculate time differences
    pairs["switch_interval"] = (
        pairs["drug_era_start_date_B"] - pairs["drug_era_end_date_A"]
    )

    # Create mask for valid transitions
    valid_mask = pairs["switch_interval"] > pd.Timedelta(0)

    # Identify drug eras with no valid subsequent eras
    no_subsequent_mask = ~pairs.groupby(["eid", "drug_era_id_A"])[
        "switch_interval"
    ].transform(lambda x: (x > pd.Timedelta(0)).any())

    no_subsequent_df = multi_era_df[
        multi_era_df["drug_era_id"].isin(
            pairs[no_subsequent_mask]["drug_era_id_A"].unique()
        )
    ].copy()
    no_subsequent_df["reason"] = "no_subsequent_era"

    # Combine all invalid cases
    invalid_df = pd.concat([single_era_df, no_subsequent_df], ignore_index=True)

    # Get valid transitions
    valid_pairs = pairs[valid_mask].copy()

    # Find minimum switch interval for each drug era A
    min_intervals = valid_pairs.groupby(["eid", "drug_era_id_A"])[
        "switch_interval"
    ].transform("min")
    valid_pairs = valid_pairs[valid_pairs["switch_interval"] == min_intervals]

    # Create final result dataframe
    result_df = pd.DataFrame(
        {
            "eid": valid_pairs["eid"],
            "A_drug_era_id": valid_pairs["drug_era_id_A"],
            "A_drug_concept_id": valid_pairs["drug_concept_id_A"],
            "A_drug_era_start_date": valid_pairs["drug_era_start_date_A"],
            "A_drug_era_end_date": valid_pairs["drug_era_end_date_A"],
            "A_drug_exposure_count": valid_pairs["drug_exposure_count_A"],
            "A_gap_days": valid_pairs["gap_days_A"],
            "B_drug_era_id": valid_pairs["drug_era_id_B"],
            "B_drug_concept_id": valid_pairs["drug_concept_id_B"],
            "B_drug_era_start_date": valid_pairs["drug_era_start_date_B"],
            "B_drug_era_end_date": valid_pairs["drug_era_end_date_B"],
            "B_drug_exposure_count": valid_pairs["drug_exposure_count_B"],
            "B_gap_days": valid_pairs["gap_days_B"],
            "switch_interval": valid_pairs["switch_interval"],
        }
    )

    return result_df, invalid_df

In [38]:
def get_drug_switches(df: pd.DataFrame) -> pd.DataFrame:
    """
    Find the closest subsequent drug era B for each drug era A.
    Process one patient at a time to reduce memory usage.
    """
    # Ensure datetime format
    if df["drug_era_start_date"].dtype != "datetime64[ns]":
        df["drug_era_start_date"] = pd.to_datetime(
            df["drug_era_start_date"], format="%d/%m/%Y"
        )
    if df["drug_era_end_date"].dtype != "datetime64[ns]":
        df["drug_era_end_date"] = pd.to_datetime(
            df["drug_era_end_date"], format="%d/%m/%Y"
        )

    # Sort all data by eid and start date
    df_sorted = df.sort_values(["eid", "drug_era_start_date"]).reset_index(drop=True)

    # List to store results for each patient
    result_dfs = []

    # Process one patient at a time
    for _, patient_df in tqdm(df_sorted.groupby("eid")):
        if len(patient_df) < 2:  # Skip patients with single drug era
            continue

        # Create indices
        n_rows = len(patient_df)
        idx_A, idx_B = np.meshgrid(np.arange(n_rows), np.arange(n_rows))

        # Convert patient_df to a dictionary of numpy arrays for faster access
        patient_arrays = {col: patient_df[col].values for col in patient_df.columns}

        # Create pairs DataFrame more efficiently
        pairs = pd.DataFrame(
            {
                "eid": patient_arrays["eid"][idx_A.flatten()],
                "drug_era_id_A": patient_arrays["drug_era_id"][idx_A.flatten()],
                "drug_concept_id_A": patient_arrays["drug_concept_id"][idx_A.flatten()],
                "drug_era_start_date_A": patient_arrays["drug_era_start_date"][
                    idx_A.flatten()
                ],
                "drug_era_end_date_A": patient_arrays["drug_era_end_date"][
                    idx_A.flatten()
                ],
                "drug_exposure_count_A": patient_arrays["drug_exposure_count"][
                    idx_A.flatten()
                ],
                "gap_days_A": patient_arrays["gap_days"][idx_A.flatten()],
                "drug_era_id_B": patient_arrays["drug_era_id"][idx_B.flatten()],
                "drug_concept_id_B": patient_arrays["drug_concept_id"][idx_B.flatten()],
                "drug_era_start_date_B": patient_arrays["drug_era_start_date"][
                    idx_B.flatten()
                ],
                "drug_era_end_date_B": patient_arrays["drug_era_end_date"][
                    idx_B.flatten()
                ],
                "drug_exposure_count_B": patient_arrays["drug_exposure_count"][
                    idx_B.flatten()
                ],
                "gap_days_B": patient_arrays["gap_days"][idx_B.flatten()],
            }
        )

        # Calculate time differences
        pairs["switch_interval"] = (
            pairs["drug_era_start_date_B"] - pairs["drug_era_end_date_A"]
        )

        # Filter valid transitions
        pairs = pairs[pairs["switch_interval"] > pd.Timedelta(0)]

        if len(pairs) == 0:  # Skip if no valid transitions
            continue

        min_intervals = pairs.groupby("drug_era_id_A")["switch_interval"].min()
        pairs = pairs.merge(
            min_intervals.rename("min_interval"),
            left_on="drug_era_id_A",
            right_index=True,
        )
        pairs = pairs[pairs["switch_interval"] == pairs["min_interval"]]

        # Create result DataFrame for this patient
        patient_result = pd.DataFrame(
            {
                "eid": pairs["eid"],
                "A_drug_era_id": pairs["drug_era_id_A"],
                "A_drug_concept_id": pairs["drug_concept_id_A"],
                "A_drug_era_start_date": pairs["drug_era_start_date_A"],
                "A_drug_era_end_date": pairs["drug_era_end_date_A"],
                "A_drug_exposure_count": pairs["drug_exposure_count_A"],
                "A_gap_days": pairs["gap_days_A"],
                "B_drug_era_id": pairs["drug_era_id_B"],
                "B_drug_concept_id": pairs["drug_concept_id_B"],
                "B_drug_era_start_date": pairs["drug_era_start_date_B"],
                "B_drug_era_end_date": pairs["drug_era_end_date_B"],
                "B_drug_exposure_count": pairs["drug_exposure_count_B"],
                "B_gap_days": pairs["gap_days_B"],
                "switch_interval": pairs["switch_interval"],
            }
        )

        result_dfs.append(patient_result)

    # Combine all results at the end
    if result_dfs:
        return pd.concat(result_dfs, ignore_index=True)
    else:
        return pd.DataFrame()  # Return empty DataFrame if no results

In [39]:
# result_df, invalid_df = get_drug_switches_all(df)
result_df = get_drug_switches(df)

100%|██████████| 281690/281690 [09:55<00:00, 472.77it/s]


In [ ]:
same_drug_switches = result_df[
    result_df["A_drug_concept_id"] == result_df["B_drug_concept_id"]
]
print(f"Number of switches between same drug: {len(same_drug_switches)}")
print(
    f"Percentage of total switches: {len(same_drug_switches) / len(result_df) * 100:.2f}%"
)

In [ ]:
print(len(df))
# print(
#     len(result_df["A_drug_era_id"].unique()) + len(invalid_df["drug_era_id"].unique())
# )
print(len(result_df["A_drug_era_id"].unique()))
# print(len(invalid_df["drug_era_id"].unique()))

In [ ]:
print(len(result_df))
# print(len(invalid_df))

test

In [68]:
df_small = df.sample(50000)

In [74]:
result_df_small = get_drug_switches(df_small)

100%|██████████| 40353/40353 [00:10<00:00, 3941.85it/s]


In [93]:
print(len(result_df_small))
print(len(result_df_small["A_drug_era_id"].unique()))

9348
9314


In [81]:
result_df_small_1, invalid_df_small_1 = get_drug_switches_all(df_small)

In [95]:
print(len(result_df_small_1))
print(len(result_df_small_1["A_drug_era_id"].unique()))

9348
9314


In [85]:
print(len(df_small))
print(
    len(result_df_small_1["A_drug_era_id"].unique())
    + len(invalid_df_small_1["drug_era_id"].unique())
)
print(len(result_df_small_1["A_drug_era_id"].unique()))
print(len(invalid_df_small_1["drug_era_id"].unique()))

50000
50000
9314
40686
